# Amazon Installations: Elastic Compute Cloud

## Usage Notes

Amazon Elastic Compute Cloud (EC2) is Amazon's service for requesting virtual machines that are created in Amazon's cloud.

This notebook will go over the different aspects of the virtual machine that you should take into consideration when you want to request a cluster of virtual machines, and the same considerations apply when requesting a single instance.

## Notebook Imports

In [ ]:
%matplotlib inline

In [ ]:
from __future__ import division, print_function
from aws_base import *
from aws_group import *
from aws_iam import *
from aws_request import *
from aws_spot import *
from aws_util import *
from aws_volumes import *
from IPython.display import display

## Identify the AMI

If you already have an AMI you want to use, set it here. The only requirement is that the AMI uses Ubuntu (otherwise, future attempts to install software won't work). If you do not set one, one will automatically be chosen for you later in the notebook.

https://console.aws.amazon.com/ec2/v2/home#Images

In [ ]:
image_id = None

## Check Spot Instance Pricing

In [ ]:
def filter_instance_types(*args):
    potential_types = [
        '%s%d.%s' % (family, x, size)
            for families, size in args
            for family in families
            for x in range(1,10)
    ]

    for extra_type in 'abcdefghijklmnopqrstuvwxyz':
        potential_types += [
            '%s%d%s.%s' % (family, x, extra_type, size)
                for families, size in args
                for family in families
                for x in range(1,10)
        ]

    candidate_types = set(potential_types)

    on_demand_prices = get_on_demand_prices()

    return [
        candidate_type for candidate_type in candidate_types
            if candidate_type in on_demand_prices or candidate_type.find('x1e.') == 0
    ]

### Consider Instance Types

It's important to consider multiple instance types for our workload, because it's not always the case that the smallest instance type is the cheapest instance type. It is not uncommon for lower tier EC2 instances prices to spike due to lazy programmers who cause the price to inflate for the lower tier EC2 instance type used by their programs. A good reference for the instance types is put together by the community of AWS users.

* http://www.ec2instances.info/

For general purpose usage, many applications are CPU-bound while still needing a decent amount of memory, so you will want to consider the general-purpose ``m`` family instances for those tasks. However, for a lot of Liferay related purposes, you can actually lean towards something more specific.

Below are a few examples of how you might decide on an instance type.

* A standard Liferay instance that you use for cluster testing is CPU-bound while needing a moderate amount of memory, and so something from the `c` family instances is appropriate.
* Liferay builds require a substantial amount of memory, but since they're running in the background, you can trade away some CPU capacity. For that reason, something from the ``r`` family is appropriate.
* Liferay upgrades require about the same amount of CPU power as a standard Liferay instance, but they need a lot more memory to succeed. When taking pricing into consideration, you would consider a larger variant in the `r` and `i` families.
* For purposes of deep learning, your only options are in the `p` family.

In [ ]:
instance_types = filter_instance_types(
#    ('c', 'large'), ('m', 'medium'),     # 6.2 cluster
#    ('c', 'xlarge'), ('m', 'large'),     # 7.0 cluster
#    ('ri', 'large'),                     # git bisect
#    ('m', 'xlarge'), ('ri', 'large'),    # normal upgrade
    ('m', '2xlarge'), ('ri', 'xlarge'),  # large upgrade
#    ('m', '4xlarge'), ('ri', '2xlarge'), # data analysis
#    ('x', 'xlarge')                      # heap dump analysis
#    ('p', 'xlarge')                      # deep learning
)

instance_types

### Plot Price History

When considering the instance type, one good metric is to look at the price of those instances types across several days. By default, the AWS GUI will show you the price history for 1 day and it allows you to look at the history for 7 days (one week) or 30 days (one month).

In [ ]:
day_count = 30

Let's plot the data so that we can compare the different instance types and then decide which instance type makes sense given the price history.

In [ ]:
target_price, on_demand_prices, instances_price_history = plot_price_history(instance_types, day_count)

## Confirm Pricing Choices

### Select Instance Type

Now that we've looked at the price history, we can choose our desired instance type.

In [ ]:
desired_instance_type = 'r5d.xlarge'

We'll make sure that we didn't make a typo.

In [ ]:
assert desired_instance_type in instance_types

If you are setting it up as a one-time test, then you should create a spot instance request (set the following variable to `False`) in order to limit the cost of running the instance.

If you wish to reuse this machine again in the future or if this is a time-consuming test where you do not want to risk early termination of the EC2 instance, you will need to be able to start and stop the instance. If that's the case, you should create an on-demand instance (set the following variable to `True`).

In [ ]:
is_on_demand = False

### Confirm Instance Type

Note that `t2` instances cannot be requested as spot instances, so we will force the value to `True` if you chose that instance type.

In [ ]:
if not is_on_demand:
    is_on_demand = not is_spot_instance_supported(desired_instance_type)

is_on_demand

Set your operating system. If you haven't chosen an `image_id`, the code is able to figure out the proper `image_id` for Amazon Linux or Ubuntu, so specify one of the two.

In [ ]:
#linux_type = 'amazon'
linux_type = 'ubuntu'
#linux_type = 'windows'

Next, the code will use this `linux_type` to figure out the user to use to connect to the virtual machine instance when running various commands.

In [ ]:
if linux_type == 'ubuntu':
    user_name = 'ubuntu'
else:
    user_name = 'ec2-user'

If you have not set an AMI, we will choose a default one based on your instance type. This choice will be made based on the virtualization options available for your instance type.

In [ ]:
if image_id is None:
    virtualization_type = get_virtualization_type(desired_instance_type)
    image_id = get_default_image_id(virtualization_type, linux_type)

image_id

The following will confirm that your instance type can be created with the virtualization type required by your AMI.

In [ ]:
ami_json = aws(
    'ec2', 'describe-images', '--image-id', image_id,
    '--region', region)

image = ami_json['Images'][0]
image_id = image['ImageId']

image_virtualization_type = image['VirtualizationType']
instance_virtualization_type = get_virtualization_type(
    desired_instance_type, image_virtualization_type)

assert image_virtualization_type == instance_virtualization_type

### Identify Availability Zone

If you are fixed to an availability zone due to an Amazon Elastic Block Store (EBS) volume, please set the availability zone containing this volume below.

In [ ]:
desired_zone_name = None

If you do not have an availability zone, we will automatically select an availability zone based on the instance type that you selected.

In [ ]:
if desired_zone_name is None:
    instance_price_history = None

    for candidate_type, candidate_price_history in instances_price_history:
        if candidate_type == desired_instance_type:
            instance_price_history = candidate_price_history
            break

    df, desired_zone_name = choose_availability_zone(
        desired_instance_type, instance_price_history, target_price)

    display(df)

desired_zone_name

### Confirm Bid Price

From our desired instance type, we will identify what our code has chosen for the bid price (essentially, it will always be the on-demand price). Note that the code below will throw an error if you made a typo in the name of your desired instance type.

In [ ]:
bid_price = on_demand_prices[desired_instance_type]
bid_price

## Create a Placement Group

In order to ensure we have good networking in our cluster, we'll need to create a placement group.

* http://docs.aws.amazon.com/AWSEC2/latest/UserGuide/placement-groups.html

For convenience, we'll need to specify the name we'll use. This name should be unique per region.

In [ ]:
placement_group_name = None

If the placement group doesn't exist, we'll create it.

In [ ]:
placement_group = get_placement_group(desired_instance_type, placement_group_name)

placement_group

## Size the Cluster

### Specify Cluster Size

Now that you've identified what kind of node you want, the next step is to identify how many you want. A single spot request can generate more than one node, and you're probably using this notebook to understand how to configure a cluster that has more than one node.

In [ ]:
cluster_size = 1

### Specify Volume Size

Cost of storage increases linearly with the number of nodes, so you may want to limit your volume size. However, chances are you have a large number of nodes because the amount of data you need to process is larger than what can fit on a standard node.

To avoid having things crash due to lacking disk space, increase this to whatever you need for your workload. Note that if you are using a `c3` or `m3` instance and you do not need the data to persist across server restarts, local storage on the machine may be able to offset some of your needs.

In [ ]:
volume_size = 20

## Confirm the Request

We now have all the details we need in order to issue our spot request.

In [ ]:
request_specification = {
    'ImageId': image_id,
    'KeyName': private_key_name,
    'SecurityGroups': security_group_names,
    'InstanceType': desired_instance_type,
    'Placement': {
        'AvailabilityZone': desired_zone_name
    },
    'BlockDeviceMappings': get_block_devices(image_id, desired_instance_type, volume_size)
}

if instance_profile_arn is not None:
    request_specification['IamInstanceProfile'] = {
        'Arn': instance_profile_arn
    }

if placement_group is not None:
    request_specification['Placement']['GroupName'] = placement_group_name

if is_on_demand:
    request_specification['DisableApiTermination'] = True
    request_specification['InstanceInitiatedShutdownBehavior'] = 'stop'

## Issue the Request

We'll make a request for our application, and to distinguish it from other requests that are cached, we'll name it `app`.

In [ ]:
if is_on_demand:
    app_request = OnDemandInstanceRequest('app')
else:
    app_request = SpotInstanceRequest('app')

app_request.request(bid_price, cluster_size, request_specification)

## Confirm Fulfillment

Here, we make sure that the request has been fulfilled and that the instances are accessible by installing `awscli` to all machines.

In [ ]:
app_instances = app_request.get_fulfilled()
app_host_names = [instance['PublicDnsName'] for instance in app_instances]

install_awscli(user_name, app_host_names)
extra_storage(user_name, app_host_names)

Next, install Docker on the virtual machine, because a lot of use cases require Docker anyway.

In [ ]:
%%writefile scripts/install_docker.sh
#!/bin/bash

if [ "" != "$(lsblk | grep -F '/mnt')" ]; then
    if [ -d /var/lib/docker ]; then
        sudo mv /var/lib/docker /mnt/
    elif [ ! -d /mnt/docker ]; then
        sudo mkdir /mnt/docker
    fi

    sudo ln -s /mnt/docker /var/lib/docker
fi

sudo apt-get install -y docker docker.io git
sudo service docker start
sudo usermod -aG docker $USER

In [ ]:
run_script(user_name, app_host_names, 'install_docker.sh')

Finally, report the host names we requested.

In [ ]:
app_host_names

### Internal Host Names

Many applications provide you with their private host name in Amazon's internal network rather than their external public host names. If you are not using SSH tunneling, this is problematic.

To alleviate this, update your hosts file to point the internal host names the external IP addresses, though you will need to clean out this file every time you create a new cluster.

Run the following lines to find out what you would need to add to your `/etc/hosts` file on Linux and Mac OS X or `/windows/system32/drivers/etc/hosts` on Windows.

In [ ]:
if assume_ssh_tunnel:
    print('You will need to use SSH tunneling to reach the services on this instance')
else:
    hosts_entries = [
        instance['PublicIpAddress'] + '\t' + instance['PrivateDnsName']
            for instance in app_instances
    ]

    print('\n'.join(hosts_entries))